# Lista Pandas - Desempenho dos Alunos GDF

O dataset analisado aqui descreve o desempenho dos alunos da rede pública do Distrito Federal. Nele encontramos as notas bimestrais de todas as matérias oferecidas a cada aluno do GDF, juntamente com informações básicas sobre a escola, o curso e a série do aluno. A fonte dos dados é o portal de dados abertos do GDF.




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import requests as r
from zipfile import ZipFile
import io

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(rows=None,download=True) -> pd.core.frame.DataFrame:
    
    """ get_data(rows: int=None, download: bool=True) -> pandas.core.frame.DataFrame:
        
        - Retorna os dados para serem utilizados na resolução da lista. 
        - Utilize o parâmetro 'download' caso você já possua os dados na sua pasta de trabalho.
        
        
        ** Parâmetros **
        ________________
        
        rows : quantidade de linhas a serem lidas no dataset, padrão é None e representa todo o dataset.
        download : define se a função executará o processo de download e extração dos dados, padrão True.
        
        ** Exemplos **
        ________________
        
        >> df = get_data(rows=100)
        >> df.shape
        >> (100,23)
        
        
        >> df = get_data(rows=578)
        >> df.shape
        >> (578,23)
        
    """
    
    if download == True:
            
        file_zipped = r.get('http://dados.df.gov.br/dataset/b8436049-44e7-4224-95b4-224718a4b166/resource/3e654a9d-0647-4e39-930e-7cd07faec888/download/dados-abertos---desempenho-escolar20180515160111.zip')  
        z = ZipFile(io.BytesIO(file_zipped.content))
        z.extractall('.')
    
    return pd.read_csv('dados abertos - desempenho escolar_20180515_160111.csv',sep=';',encoding='latin-1',nrows=rows)

In [14]:
df = get_data()

1. Quantos alunos diferentes estão matriculados no curso "Ensino Médio"?

2. Levando em consideração somente os alunos do curso "Ensino Médio". Descreva a porcentagem de alunos que foram aprovados e reprovados (aluno aprovado é aquele que tirou 5 ou mais como nota final em todas as matérias cursadas) por série.

3. As coordenações regionais são instituições que cuidam das escolas públicas de uma determinada região. Busque dados populacionais de cada coordenação regional e demonstre se há alguma correlação entra a quantidade de alunos de cada coordenação e a população daquela cidade satélite.

4. Levando em consideração somente os alunos do curso "Ensino Médio". Demonstre a porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia, por coordenação Regional.

5. Qual coordenação regional tem a maior quantidade de alunos matriculados no turno diurno do curso "Ensino Médio".Considere somente as coordenações regionais que possuem um número acima da média de aprovados.

6. Considerando somente alunos do curso "Ensino Médio". Os alunos do turno diurno perfomam melhor que os alunos do turno noturno nas matérias de química, física e biologia. (Verdadeiro ou Falso, demonstre com dados).



### Quantos alunos diferentes estão matriculados no curso "Ensino Médio"?

In [34]:
df.head()

,cod_coordenacao_regional,coordenacao_regional,cod_escola,escola,cod_curso,curso,cod_serie,serie,cod_turno,turno,cod_turma,turma,cod_aluno,bimestre,cod_disciplina,disciplina,nota_bimestral,aulas_bimestre,faltas_bimestre,faltas_justificadas_bimestre,aulas_globalizadas,faltas_globalizadas,faltas_justificadas_globalizadas
0,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,1,22,Projeto Interdisciplinar I,9.0,10.0,0.0,0.0,NaN,NaN,NaN
1,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,2,22,Projeto Interdisciplinar I,8.3,10.0,0.0,0.0,NaN,NaN,NaN
2,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,3,22,Projeto Interdisciplinar I,6.5,10.0,0.0,0.0,NaN,NaN,NaN
3,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,4,22,Projeto Interdisciplinar I,9.5,9.0,0.0,0.0,NaN,NaN,NaN
4,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,resultado final,22,Projeto Interdisciplinar I,8.5,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.curso.unique()

array(['Ensino Fundamental de 9 Anos', 'Ensino Médio',
       'Educação Infantil', 'Educação Infantil -  Especial',
       'Ensino Fundamental de 9 Anos - Especial',
       'Ensino Médio - Especial', 'Ensino Fundamental Socioeducação',
       'Ensino Médio - Socioeducação'], dtype=object)

In [12]:
# Selecionando apenas o Ensino Médio
df = df[df.curso	== "Ensino Médio"]
# Pode excluir porque é só para contar o número de alunos diferentes
df = df.drop_duplicates('cod_aluno')
alunos = len(df['cod_aluno'])
alunos

print(f' O número de alunos matriculados no Ensino Médio é: {alunos}')

 O número de alunos matriculados no Ensino Médio é: 81712


### Levando em consideração somente os alunos do curso "Ensino Médio". Descreva a porcentagem de alunos que foram aprovados e reprovados (aluno aprovado é aquele que tirou 5 ou mais como nota final em todas as matérias cursadas) por série.

In [76]:
def corrigir_nota(value):
  if value > 10:
    return value / 10
  else:
    return value

for serie in df[df.curso== 'Ensino Médio'].serie.unique():
  qtd_total_de_alunos = df[(df.curso == 'Ensino Médio')&(df.serie == serie)].cod_aluno.nunique()

  aprovados = (((
    
    df[(df.curso == 'Ensino Médio')&(df.bimestre== 'resultado final')&(df.serie == serie)]
    .loc[:,['cod_aluno','serie', 'disciplina', 'nota_bimestral']]
    .assign(nota_bimestral = lambda x: x.nota_bimestral.apply(corrigir_nota))
    .pivot_table(index='cod_aluno', columns='disciplina', values='nota_bimestral',aggfunc='max')
    .fillna(0)
  ) >= 5).sum(axis=1) >= 14).sum() # Considerando a partir de 14 disciplinas (tirando os 3 projetos)

  print(f'A porcentagem de alunos na {serie} aprovados é de {round((aprovados/qtd_total_de_alunos)*100,2)}%')
  print(f'A porcentagem de alunos na {serie} reprovados é de {round((1 - aprovados/qtd_total_de_alunos)*100,2)}%')

A porcentagem de alunos na 1ª Série aprovados é de 63.19%
A porcentagem de alunos na 1ª Série reprovados é de 36.81%
A porcentagem de alunos na 2ª Série aprovados é de 69.8%
A porcentagem de alunos na 2ª Série reprovados é de 30.2%
A porcentagem de alunos na 3ª Série aprovados é de 76.83%
A porcentagem de alunos na 3ª Série reprovados é de 23.17%


### As coordenações regionais são instituições que cuidam das escolas públicas de uma determinada região. Busque dados populacionais de cada coordenação regional e demonstre se há alguma correlação entra a quantidade de alunos de cada coordenação e a população daquela cidade satélite.

In [37]:
populacao = pd.read_html ('https://pt.wikipedia.org/wiki/Lista_de_regi%C3%B5es_administrativas_do_Distrito_Federal_por_popula%C3%A7%C3%A3o')[0]

In [41]:
# Remove o "CRE"
def remove_cre(value):
  return value[6:]

# Remove os espaços em branco e transforma str em int
def remove_whitespace(value):
  str_split = value.split()
  return int(''.join(str_split))

(
    df[['cod_aluno', 'coordenacao_regional']]
    .drop_duplicates(subset=['cod_aluno'])
    .groupby('coordenacao_regional')
    .count()
    .reset_index()
    .assign(sem_cre = lambda x: x.coordenacao_regional.apply(remove_cre)) # usando a função para remover o "CRE"
    .drop('coordenacao_regional',axis=1)
    .rename(columns={'sem_cre': 'Região administrativa','cod_aluno': 'qtd_alunos'})
    .merge(populacao[['Região administrativa','População']],on='Região administrativa', how='inner')
    .assign(População = lambda x: x.População.apply(remove_whitespace))
    .loc[:,['qtd_alunos', 'População']]
    .corr()


)

,qtd_alunos,População
qtd_alunos,1.000000,0.937452
População,0.937452,1.000000


### Levando em consideração somente os alunos do curso "Ensino Médio". Demonstre a porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia, por coordenação Regional.

In [110]:
# Corrigindo as notas que estão em padrões diferentes
def corrigir_nota(value):
  if value > 10:
    return value / 10
  else:
    return value

# Fazer um for para separar por coordenação regional
for cidade in df[df.curso == 'Ensino Médio'].coordenacao_regional.unique():
  qtd_total_de_alunos = df[(df.curso == 'Ensino Médio')&(df.coordenacao_regional == cidade)].cod_aluno.nunique()


  aprovados = (((
    df[(df.curso	== "Ensino Médio")&(df.bimestre == 'resultado final')&(df.coordenacao_regional == cidade)]
    .loc[:,['cod_aluno','coordenacao_regional','disciplina', 'nota_bimestral']]
    .assign(nota_bimestral = lambda x: x.nota_bimestral.apply(corrigir_nota))
    .pivot_table(index= 'cod_aluno', columns='disciplina', values='nota_bimestral', aggfunc='max')
    .loc[:, ['Biologia', 'Física', 'Química']]
    .fillna(0)

  ) >= 8).sum(axis=1) == 3).sum()

  print(f'A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em {cidade} é de {round((aprovados/qtd_total_de_alunos)*100,2)}%')

A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Brazlândia é de 10.74%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Ceilândia é de 14.21%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Gama é de 9.15%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Núcleo Bandeirante é de 12.15%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Planaltina é de 8.39%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Sobradinho é de 10.14%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Recanto das Emas é de 12.84%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em CRE - Samambaia é de 14.15%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química em

### Qual coordenação regional tem a maior quantidade de alunos matriculados no turno diurno do curso "Ensino Médio".Considere somente as coordenações regionais que possuem um número acima da média de aprovados.

In [18]:
# Corrigindo as notas que estão em padrões diferentes
def corrigir(valor):
    if valor>10:
        return valor/10
    else:
        return valor

# Calcula a média de aprovados
aprovados = (
        df[['coordenacao_regional','curso','bimestre','nota_bimestral','cod_aluno']]
        [df.curso == 'Ensino Médio']
        [df.bimestre == 'resultado final']
        .assign(aprovado = lambda x: x.nota_bimestral.apply(corrigir) >= 5)
        .groupby(['coordenacao_regional','cod_aluno'])
        .agg(disciplinas_aprovado=('aprovado', 'sum'))
        .assign(aprovado = lambda x: x.disciplinas_aprovado >= 14)
        .groupby('coordenacao_regional')
        ['aprovado']
        .sum()

)
print(f'A média de aprovados é {round(aprovados.mean())}\n')

# Separa as coordenações que estão acima da média
acima_da_media = (
        df
        .coordenacao_regional
        .isin(
            aprovados
            [aprovados > aprovados.mean()]
            .index
        )   
)

# Mostra apenas as coordenações que estão acima da média e a quantidade de alunos matriculados no Ensino Médio no turno diurno
acima_da_media_diurno =(
        df[acima_da_media]
        [df.turno == 'Diurno']
        [df.curso == 'Ensino Médio']
        [['coordenacao_regional', 'cod_aluno']]
        .groupby('coordenacao_regional')
        .agg(alunos=('cod_aluno', 'nunique'))
        .sort_values(by='alunos', ascending =False)
        .reset_index()

)
print(f'Coordenações Regionais que têm o número de alunos aprovados maior que a média (turno diurno):')
print(acima_da_media_diurno)

resultado = pd.DataFrame(acima_da_media_diurno)
resultado[['alunos']].values[0][0]

print(f'\nA coordenação regional com a maior quantidade de alunos matriculados no turno diurno do curso "Ensino Médio" é: {resultado.values[0][0]}')

A média de aprovados é 4017

Coordenações Regionais que têm o número de alunos aprovados maior que a média (turno diurno):
  coordenacao_regional  alunos
0      CRE - Ceilândia   12490
1     CRE - Taguatinga    9442
2   CRE - Plano Piloto    8493
3     CRE - Planaltina    5700

A coordenação regional com a maior quantidade de alunos matriculados no turno diurno do curso "Ensino Médio" é: CRE - Ceilândia


### Considerando somente alunos do curso "Ensino Médio". Os alunos do turno diurno perfomam melhor que os alunos do turno noturno nas matérias de química, física e biologia. (Verdadeiro ou Falso, demonstre com dados).

In [59]:
# Corrigindo as notas que estão em padrões diferentes
def corrigir_nota(value):
  if value > 10:
    return value / 10
  else:
    return value


# Fazer um for para separar por turno
for x in df[df.curso == 'Ensino Médio'].turno.unique():
  qtd_total_de_alunos = df[(df.curso == 'Ensino Médio')&(df.turno == x)].cod_aluno.nunique()

# Testando os aprovados
  aprovados = (((
    df[(df.curso	== "Ensino Médio")&(df.bimestre == 'resultado final')&(df.turno == x)]
    .loc[:,['cod_aluno','disciplina', 'nota_bimestral','turno']]
    .assign(nota_bimestral = lambda x: x.nota_bimestral.apply(corrigir_nota))
    .pivot_table(index= 'cod_aluno', columns='disciplina', values='nota_bimestral', aggfunc='max')
    .loc[:, ['Biologia', 'Física', 'Química']] # Seleciona apenas as 3 disciplinas
    .fillna(0)

  ) >= 5).sum(axis=1) == 3).sum()

  print(f'A porcentagem de alunos aprovados em Biologia, Física e Química no período {x} é de {round((aprovados/qtd_total_de_alunos)*100,2)}%')

# Testando a quantidade de alunos com notas maiores ou iguais a 8 
  desempenho = (((
    df[(df.curso	== "Ensino Médio")&(df.bimestre == 'resultado final')&(df.turno == x)]
    .loc[:,['cod_aluno','disciplina', 'nota_bimestral','turno']]
    .assign(nota_bimestral = lambda x: x.nota_bimestral.apply(corrigir_nota))
    .pivot_table(index= 'cod_aluno', columns='disciplina', values='nota_bimestral', aggfunc='max')
    .loc[:, ['Biologia', 'Física', 'Química']]
    .fillna(0)

  ) >= 8).sum(axis=1) == 3).sum()

  print(f'A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química no período {x} é de {round((desempenho/qtd_total_de_alunos)*100,2)}%')

print('\nFalso. Ainda que a porcentagem de alunos aprovados no período turno diurno seja superior, a diferença é pequena. Os alunos do período noturno, por sua vez, mostram um desemepenho consideravelmente superior quando analisamos as notas maiores ou iguais a 8.')

A porcentagem de alunos aprovados em Biologia, Física e Química no período Diurno é de 77.33%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química no período Diurno é de 9.54%
A porcentagem de alunos aprovados em Biologia, Física e Química no período Noturno é de 74.26%
A porcentagem de alunos com nota maior ou igual a 8 em Biologia, Física e Química no período Noturno é de 30.95%

Falso. Ainda que a porcentagem de alunos aprovados no período turno diurno seja superior, a diferença é pequena. Os alunos do período noturno, por sua vez, mostram um desemepenho consideravelmente superior quando analisamos as notas maiores ou iguais a 8.
